# Analysis-6 Design Your Own Question

Now that you’ve found the answers to the questions above, design two of your own questions to answer. These should be sufficiently difficult, and you should be creative! You should start with a question, and then propose a predicted answer or hypothesis before writing a MapReduce job to answer it. If you come up with a particularly challenging question, it can count for two (ask first).

# Prediction About Upcoming New Reddit User Numbers (Apply ML Model) 

Based on the given data set, we will find the unique user numbers in reddit for each month and then we will make prediction for upcoming month's unique user numbers. (We will use ML models) By doing this reddit development team can prepare themselves for upcoming load.

In [10]:
%%time
spark.catalog.dropGlobalTempView("Comments")
df = sqlContext.read.json("hdfs://orion11:11001/sampled_reddit_v2/*")
df.createGlobalTempView("Comments")

CPU times: user 8.32 ms, sys: 1.59 ms, total: 9.92 ms
Wall time: 25.4 s


In [11]:
%%time
botExpr = "[bB][oO][tT]"

filteredComment = (df
                   .filter(~(df.body.like("[deleted]") 
                             | df.body.like('[removed]') 
                             | df.author.rlike(botExpr) 
                             | df.author.like("[deleted]")
                            )
                          )
                  )
df.unpersist()
filteredComment.cache()
print(filteredComment.count())

27429190
CPU times: user 5.89 ms, sys: 4.7 ms, total: 10.6 ms
Wall time: 26 s


### Add month and year column into the dataset

In [12]:
%%time
from pyspark.sql.functions import year, month, dayofmonth, from_unixtime
from pyspark.sql.types import DateType

filteredComment = (filteredComment
      .withColumn("year", year(from_unixtime("created_utc").cast(DateType())))
      .withColumn("month",month(from_unixtime("created_utc").cast(DateType()))))
filteredComment.count()

CPU times: user 2.35 ms, sys: 2.16 ms, total: 4.51 ms
Wall time: 903 ms


27429190

### Count the number of active user for each month and year

In [13]:
from pyspark.sql.functions import countDistinct

userByTime = filteredComment.groupBy('month','year').agg(countDistinct('author').alias('NumOfUser'))

DataFrame[month: int, year: int, NumOfUser: bigint]

In [17]:
import pyspark.sql.functions as func
userByTime = userByTime.sort(func.asc('year'), func.asc('month'))
userByTime.show()

+-----+----+---------+
|month|year|NumOfUser|
+-----+----+---------+
|   12|2005|        7|
|    1|2006|       29|
|    2|2006|       58|
|    3|2006|       83|
|    4|2006|      129|
|    5|2006|      156|
|    6|2006|      173|
|    7|2006|      212|
|    8|2006|      310|
|    9|2006|      280|
|   10|2006|      325|
|   11|2006|      355|
|   12|2006|      393|
|    1|2007|      461|
|    2|2007|      538|
|    3|2007|      632|
|    4|2007|      671|
|    5|2007|      819|
|    6|2007|      830|
|    7|2007|      984|
+-----+----+---------+
only showing top 20 rows



In [18]:
userByTime.coalesce(1).write.format('csv').save('hdfs://orion11:11001/A6')